In [1]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows
EXISTING_CHECKPOINT_KAGGLE_DATASET_ID = "hsm-models" # set to the Kaggle dataset ID where the existing model checkpoints are stored

In [2]:
import os
import sys

from tqdm import tqdm
import torch

import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader


if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.utils.utils import get_raw_data_dir, get_submission_csv_path, get_models_save_path, set_seeds
from src.utils.constants import Constants

from src.datasets.cbramod_dataset import CBraModDataset
from src.models.cbramod_model import CBraModModel

set_seeds(Constants.SEED)

2025-11-12 19:43:11,224 :: root :: INFO :: Initialising Utils
2025-11-12 19:43:11,740 :: root :: INFO :: Initialising Datasets
2025-11-12 19:43:11,943 :: root :: INFO :: Initialising Models


Skipping module tcn due to missing dependency: No module named 'pytorch_tcn'


/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened bec

In [3]:
DATA_PATH = get_raw_data_dir()

In [9]:
class InfCFG:
    data_path = DATA_PATH
    n_splits = 5
    batch_size = 64
    num_workers = 8
    model_dir = get_models_save_path(EXISTING_CHECKPOINT_KAGGLE_DATASET_ID) / "cbramod" / DATA_PREPARATION_VOTE_METHOD
    dropout_prob = 0.1
    window_size_seconds = 5
    eeg_normalization = "naive"
    apply_preprocessing = False

InfCFG.model_paths = [os.path.join(InfCFG.model_dir, f'best_model_fold_{i}.pth') for i in range(InfCFG.n_splits)]

In [5]:
def create_model(device):
  return CBraModModel(
    pretrained_weights_path=None, # no need to load pretrained weights here as we will load full model weights later
    classifier_type="all_patch_reps",
    num_of_classes=len(Constants.TARGETS),
    dropout_prob=InfCFG.dropout_prob,
    num_eeg_channels=len(Constants.EEG_FEATURES),
    seq_len_seconds=InfCFG.window_size_seconds,
    device=device,
  ).to(device)

In [7]:
def run_inference():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    test_df = pd.read_csv(os.path.join(InfCFG.data_path, 'test.csv'))
    test_dataset = CBraModDataset(
        test_df, 
        data_path=InfCFG.data_path, 
        window_size_seconds=InfCFG.window_size_seconds, 
        eeg_frequency=200,
        mode='test',
        normalization=InfCFG.eeg_normalization,
        apply_preprocessing=InfCFG.apply_preprocessing
    )
    test_loader = DataLoader(
        test_dataset, batch_size=InfCFG.batch_size, shuffle=False, num_workers=InfCFG.num_workers
    )

    all_fold_predictions = []

    for i, path in enumerate(InfCFG.model_paths):
        print(f"\n========== Inferencing with Fold {i} Model ==========")
        if not os.path.exists(path):
            print(f"Model file not found: {path}. Skipping this fold.")
            continue
            
        model = create_model(device)
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()

        current_fold_preds = []
        with torch.no_grad():
            for eeg_windows in tqdm(test_loader, desc=f"Predicting Fold {i}"):
                outputs = model(eeg_windows.to(device))
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))

    if not all_fold_predictions:
        print("No models were found for inference. Aborting.")
        return

    avg_predictions = np.mean(all_fold_predictions, axis=0)
    submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
    submission[Constants.TARGETS] = avg_predictions

    submission.to_csv(get_submission_csv_path(), index=False)

In [10]:
run_inference()

Using device: cuda

========== Inferencing with Fold 0 Model ==========


Predicting Fold 0: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]



========== Inferencing with Fold 1 Model ==========


Predicting Fold 1: 100%|██████████| 1/1 [00:00<00:00,  3.77it/s]



========== Inferencing with Fold 2 Model ==========


Predicting Fold 2: 100%|██████████| 1/1 [00:00<00:00,  1.81it/s]



========== Inferencing with Fold 3 Model ==========


Predicting Fold 3: 100%|██████████| 1/1 [00:00<00:00,  3.89it/s]



========== Inferencing with Fold 4 Model ==========


Predicting Fold 4: 100%|██████████| 1/1 [00:00<00:00,  3.95it/s]
